In [1]:
import glob, re, sys, os, ssl, unicodedata, itertools, lxml, bs4, requests, multiprocessing
import pandas as pd
from nltk.tokenize import sent_tokenize
from dataclasses import dataclass
from pathlib import Path
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from multiprocessing.pool import Pool
#from grobid_client_python import grobid_client as grobid
from bs4 import BeautifulSoup
import boto3


##########
# This script allows for conversion of pdfs to an excel spreadsheet, parsing out text by keywords from each file
# Written/modified by Corinn Small, corinn.small@ucsf.edu
#########


def convert_text(inpath,outpath):
    
    '''
    converts text from pdf to tei.xml using grobid web service api, remember computer has to be connected to the server: cd grobid-0.6.1/ -> ./gradlew run
    input: path to papers
    output: tei.xml file per pdf
    '''

    print('Converting text...')
    print('')
    
    # make sure your aws role has the correct permissions to run textract, see https://docs.aws.amazon.com/textract/latest/dg/api-async-roles.html to configue 
    
    client = boto3.client('textract')
    for m in dir(client): print(m)  # what are the function attached to the client object?... _ = 'private' function meaning dont call it
    
    
    client.StartDocumentTextDetection()  #for detecting and analyzing text in multipage docs (asynchronous op), I would try both syncrhonous and asynchronous, maybe the synchronous works for pdfs with just a couple of pages?

#have to set up a aws account s3 bucket... (cloud object storage)
#for s3object, you need to specify bucket, name and version:
    s3_obj =  {"Bucket": BUCKET, "Name": NAME}  #creating dictionary obj

    response = client.GetDocumentTextDetection(Document = {'S3Object': s3_obj})  #nesting dictionary s3_obj inside document dictionary and giving that to the client function 
#get results returned, we want an s3object dictionary to be returned 

#response is a dictionary type: you can look at the document page that specifies the basic the s3 object metadata to get fields that you can query from the dict

    detect = response.copy()  #save output as detection (of the pdf output)

#now to analyze all the gibberish (output = list of block objects) this is the synchronous version:
    response2 = client.analyze_document(Document = {'S3Object': s3_obj}, FeatureTypes = ['TABLES'])
#document = s3object
#observation: of required syntax: the layout is confusing to you, the request syntax that the boto3 doc specifies is telling you what type of ojbect to pass the client.function and the naming convention of the returned object! thats it! dont worry, just keep in mind the indendation and parentheses
#featuretypes is required
    response2['Blocks'][0].keys()  # print name of each block

    for b in response2['Blocks']:
        if b['BlockType'] == 'LINE':
            print("{}\t{}".format(b['Text'], b['Confidence']))  # for everytime line is found, what is that word and the confidence at which its identified, it's important/critical to check the confidence scores!



    
    #client = grobid.grobid_client(config_path="./grobid_client_python/config.json")
    #client.process("processFulltextDocument", inpath, outpath)
    
    print('Done')

def read_tei(tei_file):
    '''
    Reads in an xml file and returns a beautifulsoup object
    
    '''
    with open(tei_file, 'r') as tei:
        soup = BeautifulSoup(tei, 'html.parser')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')
    

def elem_to_text(elem, default='NA'):
    '''
    Returns element if it exists, if not returns NA
    
    '''
    if elem:
        return elem.getText()
    else:
        return default
    

#create class for storing pdf info

class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)  #creates soup object
        self._text = None
        self._title = ''  
        self._abstract = ''
        self._keytext = {}
        self._check = None
        
    @property
    def doi(self, id_='DOI'):
        '''
        retrieve id
        '''
    
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return 'no id'
        else:
            return idno_elem.getText()
    
    @property
    def title(self):
        '''
        retrieve title
        '''
        if not self._title:
            self._title = self.soup.title.getText()
        return self._title
    
    @property
    def abstract(self):
        '''
        retrieve abstract
        '''
        
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True)
            self._abstract = abstract
        return self._abstract
      
    @property
    def authors(self):
        '''
        retrieve authors
        '''
        authors_in_header = self.soup.analytic.find_all('author')

        result = []
        
        @dataclass
        class Person:
            firstname: str
            middlename: str
            surname: str
                
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename", type="first"))
            middlename = elem_to_text(persname.find("forename", type="middle"))
            surname = elem_to_text(persname.surname)
            person = Person(surname, firstname, middlename)
            result.append(person)
        return result
    
    @property
    def text(self):
        '''
        retrieves text
        returns dictionary by subsection
        '''
        #print(self.soup.prettify())
        print(self.filename.split('/')[-1])
        print('')
        
        if not self._text:
            paper_text = {}  
                
            divs = self.soup.body.find_all('div')
            #print(divs)
                
            for div in divs:   
                if not div.get('type'):  # if div is neither an appendix nor reference
                    heads = div.find_all('head')  #find all subsections
                    #print(heads)
                    
                    if not heads:  #if there aren't any subsections
                        sect = []
                        for p in div.find_all('p'):  #add all sentences to one section titled 'body'
                            #print(p)
                            sect_text = p.get_text(separator=' ', strip=True)
                            sect.append(sect_text)
                            paper_text['body'] = sect  
                        
                        #print(sect)
                        
                    else:  #otherwise for each subsection create a new list with corresponding text and add it to a div dictionary
                        for head in heads:
                            sect_ = []
                            head_text = head.get_text(separator=' ', strip=True).lower()
                            
                            
                            if len(head_text.split(' ')) > 7:  #if the head title is < 7 words long (arbitrary #) ask the user to clarify its validity
                                print('subtitle sentence: ', head_text)
                                
                                ans = input('Is this subsection a full sentence? y/n')
                                ok_ans = ('y','n')
                                
                                while ans not in ok_ans:
                                    print('Y or N only please!')
                                    ans = input('Is this subsection a full sentence? y/n')

                                if ans == 'y':  #if the title is a sentence
                                    print('Including subsection title in searchable text, but still include it as a separate subsection...')
                                    print('')
                                    sect_.append(head_text)  #include it in the text
                                    for p in div.find_all('p'):  #for each paragraph find all text 
                                        #print(p)
                                        sect_text = p.get_text(separator=' ', strip=True)
                                        sect_.append(sect_text)  #add it to the list


                                elif ans == 'n':  #if the title is not a sentence
                                    ans1 = input('Does this subsection make sense? y/n')  #does the title make sense in general?

                                    while ans1 not in ok_ans:
                                        print('Y or N only please!')
                                        ans1 = input('Is this subsection a full sentence? y/n')


                                    if ans1 == 'y':  #if the title makes sense treat it as a regular subsection
                                        print('Valid subsection...')
                                        print('')

                                        for p in div.find_all('p'):
                                            #print(p)
                                            sect_text = p.get_text(separator=' ', strip=True)
                                            sect_.append(sect_text)

                                    elif ans1 == 'n':  #if the title doesn't make sense, mark self._check True so the user knows to check it later
                                        print('Flagging nonsense...')
                                        print()

                                        self._check = True

                                        for p in div.find_all('p'):  #still include the subsection's text in list for keyword searching
                                            #print(p)
                                            sect_text = p.get_text(separator=' ', strip=True)
                                            sect_.append(sect_text)

                                    

                            else:  #if the subsection title is not longer than 7 words, find all text and add it to the list
                                for p in div.find_all('p'):
                                    #print(p)
                                    sect_text = p.get_text(separator=' ', strip=True)
                                    sect_.append(sect_text)
                                    #print('subsection title < 7 words')
                        
                                
                            
                            paper_text[head_text] = sect_ 
                                    
                                    
                        #for figure descriptions....(WORK IN PROGRESS)
                            if head_text.find('fig') != -1:  #if figure is treated as its own subsection, find all text and include it in its own list
                                print('Figure treated as subsection: ', head_text)
                    
                                for p in div.find_all('p'):
                                    #print(p)
                                    sect_text = p.get_text(separator=' ', strip=True)
                                    sect_.append(sect_text)
                                    paper_text['figure'] = sect_
                                
                            for fig in self.soup.body.find_all('figure'):   #for figure sections find the figure descriptions and print it
                                figDesc = fig.find_all('figDesc')
                                for f in figDesc:
                                    print('Figure Description: ',f)
                                           
                  
                
            #if paper is short, all the text might show up in the abstract section of the xml file instead of div
            if not divs:
                self._check = True
                
                sect = []
                ps = self.soup.abstract.find_all('p')
                
                if not ps:
                    sect_text = self.soup.abstract.get_text(separator=' ', strip=True)
                    sect.append(sect_text)


                else:
                    for p in ps:
                        sect_text = p.get_text(separator=' ', strip=True)
                        sect.append(sect_text)

                paper_text['body'] = sect   #add all sentences to one section titled 'body'
                
                    
                
                
                
            self._text = paper_text
        return self._text
    
    

    
    def keytext(self, keywords):
        '''
        retrieves sentences by keyword
        returns dictionary
        '''
        
        
        for keyword in keywords:
            self._keytext[keyword] = []
            
        
        for k,v in self._text.items():  #for subsection and text 
                for keyword in keywords:  #search thru sentences 
                    sub_dict = {}
                    section = k.lower()
                    sub_dict[section] = []  #create list for each subsection in dictionary
                    
                    for i in v:  #for paragraph in text
                        sentences = sent_tokenize(i)  #get list of sentences
                        sentences = [s.lower() for s in sentences]
                        
                    
                        for sentence in sentences:
                            result = re.findall('\\b' + keyword + '\\b', sentence)  #find keyword in sentence

                            if len(result) > 0:  #if keyword exists, 
                                sub_dict[section].append(sentence)  #add sentence to subsection list

                            else:  
                                pass
                    
                    #print(sub_dict)
                    self._keytext[keyword].append(sub_dict)  #adds each subsection to keyword dictionary
                        
        return self._keytext                   

In [25]:
#get input from user

cases = 'summary'
path = '/Users/corinnsmall/Documents/BPDCN/bpdcn_papers/' + cases +'_case_papers/'
key = 'wgs'
output = 'output_' + cases + '_cases_' + key

associated_diseases_keywords = ['acute myeloid leukemia', 'AML', 'acute lymphoblastic leukemia', 'ALL', 'leukemia', 'non-hodgkin lymphoma', 'Hodgkin lymphoma',
                                'lymphoma','myelogenous leukemia', 'multiple myeloma', 'myeloma', 'chronic myelogenous leukemia', 'CML', 'chronic lymphocytic leukemia', 
                                'CLL', 'carcinoma','Pleuropulmonary blastoma', 'blastoma', 'neuroblastoma', 'melanoma', 'sarcoma', 'skin cancer', 'hairy cell leukemia',
                                'ependymoma', 'chordoma','bone cancer', 'bladder', 'AIDS-related lymphoma', 'thyroid cancer', 'colon cancer', 'rectal cancer',
                                'prostate cancer', 'chronic myeloid leukemia', 'myeloproliferative', 'myelodysplastic', 'mast cell', 'mastocytosis', 'lymphoblastic', 
                                'follicular lymphoma', 'marginal zone lymphoma', 'langerhan', 'polycythemia vera', 'essential thrombocythemia', 'myelofibrosis', 
                                'mycosis fungoides', 'sezary', 'burkitt', 'cmml', 'chronic myelomonocytic leukemia']

morphology_keywords = ['Vacuoles','Vacuolated','Microvacuoles','Lymphoid','Eccentrically','Eccentric','Prominent nucleoli', 'Small nucleoli','Large nucleoli',
                       'Medium nucleoli','azurophilic','Blast','Blastoid','Agranular','Basophilic','Eosinophilic','Perivascular','Periadnexal','Pseudopodia','Hairy','Rosary beads',
                       'Large nucleolus','Monoblastic','Monocytic','Histiocytic','Histiocytoid','Small granules','Large granules','Granulated','Granular','Condensed chromatin',
                       'Dispersed chromatin','Fine chromatin','Pale cytoplasm','Poorly differentiated', 'Large sized','Medium sized','Small sized','Plasmablast', 
                       'Plasmacytoid','plasmacytic','Immature','lymphoblast']

nuclei_keywords = ['convoluted nuclei','convoluted','convolutions','nuclear folds','folds','membrane irregularities','slightly irregular','round nuclei','oval nuclei','vesicular chromatin','cleaved nuclei']


gene_keywords = ['ABL1','AKT1','ALK','APC','ARF','ARID1A','ASH1L','ASXL1','ASXL3','ATM','ATR','AXIN2','BRAF','BCORL1','CAL-1','CCND3','CEBPA','CHD8','CHP2','CDH1','CDKN2A',
                 'CDKN1B','CDKN2B','CIC','CREBBP','CSF1R','CTNNB1','CPS1','CROCC','CXCR4','DAXX','DIP2A','DNMT3A','EGFR','EGR1','EP300','ERBB2','ERBB4','ERCC4','ETV6','EYA2',
                 'EZH2','FBXW7','FGFR1','FGFR2','FGFR3','FLT3','FLT3-ITD','FLT3-other','GNA11','GNAS','GNAQ','GPR160','HES6','HNF1A','HOXB9','HRAS','IDH1','IDH2','IKZF1',
                 'IKZF2','IKZF3','IVL','JAK2','JAK3','KDM40','KDR','KIT','cKIT','KRAS','MAD1L1','MAPK1','MCL1','MET','MLH1','MLL','MLL2','MLL3','MPL','MSH6','MYB','MYBL1',
                 'MYC','MYST3','MYST4','NF1','NOTCH1','NPM1','NR3C1','NRAS','PALB2','PARK2','PBRM1','PDGFRA','PHF2','PHF6','PIK3CA','PMDC05','PLCXD3','PLP1','PTEN','PTPN11',
                 'PTPN23','PVT1','RAD52','RANBP2','RAS','RB1','RET','RFPL1','RHOA','RUNX1','RUNX2','SARDH','SIGLEC6','SLC25A10','SMAD4','SMARCB1','SMARCD1','SMO','SRC','SRCAP',
                 'SRSF2','STK3','STK11','SUPT3H','SUZ12','TCF3','TCL1A','TEL','TET2','TRMT61B','TP53','U2AF1','UBE2G2','VHL','WNT3','WNT7B','WNT10A','WT1','ZEB2','ZRSR2']

wgs_keywords = ['whole exome sequencing', 'whole genome sequencing', 'wes', 'wgs', 'sequencing', 'exome', 'whole genome', 'whole-genome', 'whole-exome', 'genome-wide']




keylist = [i.lower() for i in wgs_keywords]  #make all keywords lower case

In [16]:
#convert pdfs to xml, need to connect to grobid api to use convert_text()

path = '/Users/corinnsmall/Documents/BPDCN/bpdcn_papers/'
outpath = path + 'xml_output/'
convert_text(path,outpath)

Converting text...

GROBID server does not appear up and running, the connection to the server failed


NameError: name 'exit' is not defined

In [26]:
#get .xml files from path, convert pdf to TEIFILE object
#get dictionary of keywords and sentences
#create formatted lists for creation of dataframe


files = glob.glob(path + 'xml_output/' + '*.xml')
#files = glob.glob('/Users/corinnsmall/Documents/BPDCN/bpdcn_papers/example/output/*.xml')

files_list = []


for file in files:
    #print(file)
    file_list = []
    f = TEIFile(file)
    name = f.filename.split('/')[-1].split('.')[0]
    f.text
    print('-----------------------------------------------------------')
    print('')
    
    
    f.keytext(keylist)
    
    for k,v in f._keytext.items():
        for d in v:
            #print('subsection dict: ', d)
            for i,j in d.items():
                #print(i, '\n', j)
                
                if len(j) == 0 and f._check == None:
                    file_list = [name, f.filename, f.doi, 'NA', k, i, 'NA']

                elif f._check == None:
                    file_list = [name, f.filename, f.doi, 'NA', k, i, j]   

                else:
                    file_list = [name, f.filename, f.doi, f._check, k, i, j]

                files_list.append(file_list)
   

laribi_2014.tei.xml

-----------------------------------------------------------

leclerc_2017.tei.xml

-----------------------------------------------------------

hamadeh_2019.tei.xml

-----------------------------------------------------------

martinmartin_2015.tei.xml

subtitle sentence:  immunophenotypic subgroups of blastic plasmacytoid dendritic cell neoplasms


Is this subsection a full sentence? y/n y


Including subsection title in searchable text, but still include it as a separate subsection...

subtitle sentence:  clinical and laboratory features of the distinct maturation-associated subgroups of bpdcn


Is this subsection a full sentence? y/n y


Including subsection title in searchable text, but still include it as a separate subsection...

-----------------------------------------------------------

jaye_2006.tei.xml

subtitle sentence:  bdca-2 is expressed on a subset of cd4 þ cd56 þ hematodermic neoplasms


...

Including subsection title in searchable text, but still include it as a separate subsection...

subtitle sentence:  c a s e r e p o r t s


Is this subsection a full sentence? y/n y


Including subsection title in searchable text, but still include it as a separate subsection...

subtitle sentence:  r e v i e w o f c a s e s p u b l i s h e d i n t h e l i t e r a t u r e


Is this subsection a full sentence? y/n y


Including subsection title in searchable text, but still include it as a separate subsection...

-----------------------------------------------------------



In [24]:
#create dataframe and write to excel file

df = pd.DataFrame(files_list, columns = ['pdf_name','pdf_location','doi', 'check_paper_content','keywords', 'section', 'text'])
                
try:
    with open(path + output + '.xlsx', 'wb') as out:
        df.to_excel(out)
        
    if out.closed:
        print('Data Ahoy!')

except IOError:
        print('I/O error')

Data Ahoy!
